In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np





dataset_dir = r"C:\Users\A\Desktop\datasets\caltech-101-img"



imagedatagenerator=ImageDataGenerator(rescale=1.0/255)
batch_size = 2000





dataset_generator =imagedatagenerator.flow_from_directory(
    dataset_dir,
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'
)





x_train, y_train =  dataset_generator[0]
x_test, y_test = dataset_generator[1]

print(len(x_train))
print(len(x_test))




# Load VGG16 without top layers
weights_path = r"C:\Users\A\Desktop\datasets\vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_model = VGG16(weights=weights_path, include_top=False, input_shape=(64, 64, 3))





for layer in base_model.layers:   #freeze the weights
   layer.trainable = False






model=keras.Sequential([
    base_model,
    keras.layers.Flatten(),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dense(102,activation='softmax'),
])

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])






# Train the model
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))





import matplotlib.pyplot as plt
predicted_value = model.predict(x_test)




from tensorflow.keras.optimizers import Adam








# unfreeze last 4 layers of base model
for layer in base_model.layers[len(base_model.layers) - 4:]:
   layer.trainable = True
    
model=keras.Sequential([
    base_model,
    keras.layers.Flatten(),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dense(102,activation='softmax'),
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# training fine tuned model
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))







import matplotlib.pyplot as plt
predicted_value = model.predict(x_test)






labels = list(dataset_generator.class_indices.keys())
n = 24
plt.imshow(x_test[n])
print("Preditcted: ",labels[np.argmax(predicted_value[n])])
print("Actual: ", labels[np.argmax(y_test[n])])








